In [24]:

import string
alphabet = list(string.ascii_lowercase)
import numpy as np
import numpy as np
import torch

from pymoo.core.problem import ElementwiseProblem

from botorch.acquisition import AcquisitionFunction
from pymoo.core.mixed import MixedVariableGA
from pymoo.core.variable import Real, Integer, Choice, Binary
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.mixed import (
    MixedVariableMating, 
    MixedVariableGA, 
    MixedVariableSampling, 
    MixedVariableDuplicateElimination
)
from pymoo.core.population import Population
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.config import Config
Config.show_compile_hint = False

from olympus.surfaces import Surface


In [ ]:
def set_pymoo_param_space(param_space):
    """ convert Olympus parameter space to pymoo 
    """
    pymoo_space = {}
    for param in param_space:
        if param.type == 'continuous':
            pymoo_space[param.name] = Real(bounds=(param.low,param.high))
        elif param.type == 'discrete': 
            # TODO: need to map the discrete params to an integer
            quit()
        elif param.type == 'categorical':
            quit()

    return pymoo_space

class PymooProblemWrapper(Problem):
    def __init__(self, surface, vars):
        super().__init__(vars = vars, n_obj = 1, n_constr = 0)
        self.surface = surface 
        
    def _evaluate(self, params: np.ndarray, out: dict, *args, **kwargs):
        
        vals = self.surface.run(params)
        
        out['F'] = np.array(vals)
        
        
# def get_init_pop(space, pop, i)

In [ ]:
pymoo_space = set_pymoo_param_space(surface.param_space)

In [ ]:
problem = PymooProblemWrapper(surface, pymoo_space)

## Very simple pymoo problem from debugging

In [30]:
def known_constraint(params): 
    if params[3] < 0.04:
        return False
    
    if params[1]=='nothing' and params[4] < 3:
        return False
    
    return True

class MixedVariableProblem(Problem):

    def __init__(self, known_constraint, **kwargs):
        vars = {
            #"b": Binary(),
            "x": Choice(options=alphabet),
            "y": Choice(options=alphabet),
            "z": Choice(options=alphabet),

            #"y": Integer(bounds=(0, 2)),
            #"z": Real(bounds=(0, 5)),
            #"a": Real(bounds=(0, 5)),
        }
        super().__init__(
            vars=vars, 
            n_var=len(vars),
            n_obj=1, 
            #n_constr=1,
            **kwargs,
        )
        
        self.known_constraint = known_constraint
        

    def _known_constraint_wrapper(self, params):
        if self.known_constraint(params):
            return -1.
        else:
            return 1.

    def _evaluate(self, X, out, *args, **kwargs):
        
        # for 'Problem', X is a list of dictionaries
    
        # objective function
        #b, x, y, z, a = X[0], X[1], X[2], X[3], X[4]
    
        f = []
        for X_ in X:
            f_ = np.random.uniform()
            f.append(f_)
        f = np.array(f)
            
        # known constraints
#         g = []
#         for X_ in X:
#             g_ = self._known_constraint_wrapper(list(X_.values()))
#             g.append(g_)
#         g = np.array(g)
        

        out["F"] = f
#         out["G"] = g

In [31]:
problem = MixedVariableProblem(known_constraint=known_constraint)
algorithm = MixedVariableGA(pop=100)

res = minimize(
    problem, 
    algorithm, 
    termination=('n_evals', 1000),
    seed=1,
    verbose=True,
    save_history=True
)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       50 |  0.5107306605 |  0.0028703270
     2 |      100 |  0.2568863115 |  0.0028703270
     3 |      150 |  0.1731065037 |  0.0028703270
     4 |      200 |  0.1310231110 |  0.0028703270
     5 |      250 |  0.1061821146 |  0.0028703270
     6 |      300 |  0.0915987869 |  0.0028703270
     7 |      350 |  0.0728304402 |  0.0007409130
     8 |      400 |  0.0644990916 |  0.0007409130
     9 |      450 |  0.0623303984 |  0.0007409130
    10 |      500 |  0.0519409044 |  0.0007409130
    11 |      550 |  0.0433818606 |  0.0007409130
    12 |      600 |  0.0428039735 |  0.0007409130
    13 |      650 |  0.0347229168 |  0.0007409130
    14 |      700 |  0.0306265877 |  0.0001710498
    15 |      750 |  0.0284589407 |  0.0001710498
    16 |      800 |  0.0265800666 |  0.0001710498
    17 |      850 |  0.0253995495 |  0.0001710498
    18 |      900 |  0.0239145088 |  0.0001710498
    19 |      950 |  0.0223046406 |  0.0001710498


Population([<pymoo.core.individual.Individual object at 0x28c22d600>,
           dtype=object)

In [6]:
res.exec_time

0.21643400192260742

In [22]:
type(res.pop[0].X['z'])

numpy.float64

In [33]:
for ind in res.pop:
    print(ind.X)
    print(ind.F)
    print('-'*25)

{'x': 'u', 'y': 's', 'z': 'v'}
[0.00017105]
-------------------------
{'x': 's', 'y': 'h', 'z': 'v'}
[0.00074091]
-------------------------
{'x': 'u', 'y': 'r', 'z': 'b'}
[0.00094054]
-------------------------
{'x': 'v', 'y': 'b', 'z': 'b'}
[0.00224965]
-------------------------
{'x': 'b', 'y': 'l', 'z': 'y'}
[0.00287033]
-------------------------
{'x': 'n', 'y': 'y', 'z': 'u'}
[0.00379483]
-------------------------
{'x': 'q', 'y': 'x', 'z': 'k'}
[0.00509351]
-------------------------
{'x': 'h', 'y': 'b', 'z': 'u'}
[0.00736889]
-------------------------
{'x': 'u', 'y': 'j', 'z': 'b'}
[0.00809394]
-------------------------
{'x': 'n', 'y': 'f', 'z': 'q'}
[0.00950379]
-------------------------
{'x': 'y', 'y': 'd', 'z': 'm'}
[0.01040431]
-------------------------
{'x': 's', 'y': 'g', 'z': 'p'}
[0.01051401]
-------------------------
{'x': 'z', 'y': 'r', 'z': 'm'}
[0.01221286]
-------------------------
{'x': 'v', 'y': 'l', 'z': 'a'}
[0.01237029]
-------------------------
{'x': 'a', 'y': 'p',

## Fully continuous example problem

In [12]:
def known_constraint(params): 
    if params[3] < 0.04:
        return False
    
    if params[1]=='nothing' and params[4] < 3:
        return False
    
    return True

class FullyContinuousProblem(Problem):

    def __init__(self, known_constraint, **kwargs):
        vars = {
            "y": Real(bounds=(0, 2)),
            "z": Real(bounds=(0, 5)),
            "a": Real(bounds=(0, 5)),
        }
        super().__init__(
            vars=vars, 
            n_var=5,
            n_obj=1, 
            n_constr=1,
            **kwargs,
        )
        
        self.known_constraint = known_constraint
        

    def _known_constraint_wrapper(self, params):
        if self.known_constraint(params):
            return -1.
        else:
            return 1.

    def _evaluate(self, X, out, *args, **kwargs):
        
        # for 'Problem', X is a list of dictionaries
    
        # objective function
        #b, x, y, z, a = X[0], X[1], X[2], X[3], X[4]
    
        f = []
        for X_ in X:
            f_ = X_['z'] + X_['y']
#             if X_['b']:
#                 f_ = 100 * f_
#             if X_['x'] == "multiply":
#                 f_ = 10 * f_
            f.append(f_)
        f = np.array(f)

        out["F"] = f

## Constrained optimization problem

In [24]:

class ConstrainedProblem(ElementwiseProblem):

    def __init__(self, **kwargs):
        super().__init__(n_var=2, n_obj=1, n_ieq_constr=1, n_eq_constr=0, xl=0, xu=2, **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x[0] ** 2 + x[1] ** 2
        out["G"] = 1.0 - (x[0] + x[1])


In [ ]:
problem = MixedVariableProblem()
algorithm = MixedVariableGA(pop=10)

res = minimize(
    problem, 
    algorithm, 
    termination=('n_evals', 1000),
    seed=1,
    verbose=True,
    save_history=True
)